In [2]:
!pip install https://github.com/kpu/kenlm/archive/master.zip 

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Installing build dependencies ... done
  Running setup.py bdist_wheel for kenlm ... done
  Stored in directory: /scratch/pbs.4608544.kman.restech.unsw.edu.au/pip-ephem-wheel-cache-_5ejv0gj/wheels/2d/32/73/e3093c9d11dc8abf79c156a4db1a1c5631428059d4f9ff2cba
Successfully built kenlm
jupyter-core 5.2.0 requires platformdirs>=2.5, which is not installed.
jupyter-client 8.0.2 requires importlib-metadata>=4.8.3; python_version < "3.10", which is not installed.
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.36 which is incompatible.
You are using pip version 10.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
pip install pyctcdecode==0.3.0

In [ ]:
import librosa
import numpy as np

In [ ]:
data_train_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_spontaneous_1050_1150_1075_228_task3_test.csv'
data_test_fp = '/srv/scratch/z5313567/thesis/AusKidTalk_local/AusKidTalk_spontaneous_1050_1150_1075_228_task3_test.csv'
model_fp = '/srv/scratch/z5313567/thesis/wav2vec2/model/AusKidTalk/finetune_20230708_2'

In [ ]:
from datasets import load_dataset

dataset = load_dataset('csv', 
                    data_files={'train': data_train_fp,
                                'test': data_test_fp})

In [ ]:
dataset

In [ ]:
def speech_file_to_array_fn(batch):
    #speech_array, sampling_rate = sf.read(batch["filepath"])
    speech_array, sampling_rate = librosa.load(batch['filepath'], sr=16000)
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["transcription_clean"]
    return batch

data = dataset.map(speech_file_to_array_fn, remove_columns=dataset.column_names["train"], num_proc=4)

In [ ]:
data

In [ ]:
import IPython.display as ipd

audio_sample = data["test"][2]
print(audio_sample["target_text"].lower())
ipd.Audio(data=audio_sample["speech"], autoplay=True, rate=audio_sample["sampling_rate"])

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained(model_fp)
model = Wav2Vec2ForCTC.from_pretrained(model_fp)
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
#processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-100h")
#model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
inputs = processor(audio_sample["speech"], sampling_rate=16_000, return_tensors="pt")

In [ ]:
import torch

with torch.no_grad():
  logits = model(**inputs).logits

In [ ]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

transcription[0].lower()

In [1]:
from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm")

/home/z5313567/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 15635.80it/s]


NameError: name 'kenlm' is not defined

In [ ]:
logits.shape

In [ ]:
" ".join(sorted(processor.tokenizer.get_vocab()))

In [ ]:
pred_ids = np.argmax(logits, axis=-1)
transcription = processor.batch_decode(pred_ids)
transcription[0].lower()